# Построение моделей для предсказания показателей регионов

## Описание работы

В данной работе предпринимается попытка создания моделей, позволяющих предсказывать:
* Контроль над ходом госзакупок (доля отмененных конкурсов в общем количестве), %;
* Общественное обсуждение законопроектов в Интернете (да - 1, нет - 0);
* Количество зарегистрированных организаций ТОС на 10 тысяч человек, штук.

Для предсказания используются следующие индикаторы:
* Институционные факторы:
    1. elections - Конкурентность выборов (да - 1, нет - 0);
    2. parliament - Уровень парламентской конкуренции, %;
    3. executive - Уровень конкуренции при формировании исполнительной власти, %;
    4. citizens_election - Включенность граждан в избирательный процесс, %;
    5. citizens_org - Участие граждан в деятельности общественных организаций, %;
* Инфраструктурные факторы:
    1. internet_all - Количество абонентов сети интернет, тысяч;
    2. internet_mobile - Количество абонентов мобильного интернета на 100 человек, единиц;
    3. internet_pc - Число персональных компьютеров на 100 человек, штук;
    4. org_access - Организации, связанные с бизнесом и использующие интернет (от общего числа обследованных организаций), %;
    5. org_site - Организации, связанные с бизнесом и имеющие веб-сайт (от общего числа обследованных организаций), %;
    6. org_pc - Число персональных компьютеров на 100 работников в бизнесе, штук;
    7. edm - Системы электронного документооборота (от общего числа обследованных организаций), %;
    8. edm_external - Автоматический обмен данными между своими и внешними информационными системами (от общего числа обследованных организаций), %;
    9. authority_access - Организации, имеющие отношение в органам власти и использующие интернет (от общего числа обследованных организаций), %;
    10. public_services - Доступность госуслуг, в том числе и за счет сокращения сроков предоставления (да - 1, нет - 0);
    11. open_data - Наличие инфраструктуры открытых данных, в том числе государственных (да - 1, нет - 0);
    12. open_election - Открытость процесса выборов (да - 1, нет - 0);
* Ресурсные факторы:
    1. average_edu - Доля населения со средним образованием, %;
    2. high_edu - Доля населения с высшим образованием, %;
    3. degree - Доля населения, имеющего ученые степени, %;
    4. ict - Доля специалистов в области ИКТ, %;
    5. grp - ВРП на душу населения, рублей;
    6. income - Среднедушевые доходы населения, рублей;
    7. po - Объем использования программного обеспечения, %;
    8. invest - Удельный вес инвестиций в основной капитал в ВВП, %;
    9. venture - Доступность венчурного капитала, штук;
    10. pc - Количество персональных компьютеров на 100 человек, штук;
    11. nt - Затраты организаций на сетевые технологии, миллионов рублей; 
    12. ict_grp - Удельный вес затрат на ИКТ в ВРП, %.
    
Необходимо отметить, что в работе использовался достаточно скудный набор данных, из-за чего модели могут существенно отличаться при новом, более обширном анализе. С учетом этого, большинство шагов автоматизировано, что позволяет пересчитывать параметры моделей без внесения изменений в код.

## Код с комментариями

Импортируем используемые библиотеки.

In [1]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sb
from scipy import stats
from sklearn import preprocessing
from IPython.display import display
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn import decomposition
from sklearn import linear_model
from sklearn import ensemble
import matplotlib.pyplot as plt

ImportError: C extension: /home/ak/.virtualenvs/magister/lib/python2.7/site-packages/pandas/hashtable.so: undefined symbol: PyFPE_jbuf not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext --inplace' to build the C extensions first.

Настраиваем графики для отображения в теле блокнота без скролла.

In [ ]:
%matplotlib inline

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

Инициализируем глобальные константы - предельный уровень корреляции, список непрерывных типов данных, метод измерения качества модели, списки классификаторов и регрессоров.

In [ ]:
CORRELLATION_LIMIT = 0.85
NUMERIC_TYPES = ['float16', 'float32', 'float64', 'complex64', 'complex128']
SCORING_CLF = 'r2'
SCORING_RGR = 'r2'
CLASSIFIERS = [linear_model.LogisticRegression, ensemble.GradientBoostingClassifier]
REGRESSORS = [linear_model.Ridge, ensemble.GradientBoostingRegressor]

Создадим словарь с расшифровками имен факторов:

In [ ]:
FACTOR_NAMES = {
    'elections': u'Конкурентность выборов (да - 1, нет - 0)',
    'parliament': u'Уровень парламентской конкуренции, %',
    'executive': u'Уровень конкуренции при формировании исполнительной власти, %',
    'citizens_election': u'Включенность граждан в избирательный процесс, %',
    'citizens_org': u'Участие граждан в деятельности общественных организаций, %',
    'internet_all': u'Количество абонентов сети интернет, тысяч',
    'internet_mobile': u'Количество абонентов мобильного интернета на 100 человек, единиц',
    'internet_pc': u'Число персональных компьютеров на 100 человек, штук',
    'org_access': u'Организации, связанные с бизнесом и использующие интернет (от общего числа обследованных организаций), %',
    'org_site': u'Организации, связанные с бизнесом и имеющие веб-сайт (от общего числа обследованных организаций), %',
    'org_pc': u'Число персональных компьютеров на 100 работников в бизнесе, штук',
    'edm': u'Системы электронного документооборота (от общего числа обследованных организаций), %',
    'edm_external': u'Автоматический обмен данными между своими и внешними информационными системами (от общего числа обследованных организаций), %',
    'authority_access': u'Организации, имеющие отношение в органам власти и использующие интернет (от общего числа обследованных организаций), %',
    'public_services': u'Доступность госуслуг, в том числе и за счет сокращения сроков предоставления (да - 1, нет - 0)',
    'open_data': u'Наличие инфраструктуры открытых данных, в том числе государственных (да - 1, нет - 0)',
    'open_election': u'Открытость процесса выборов (да - 1, нет - 0)',
    'average_edu': u'Доля населения со средним образованием, %',
    'high_edu': u'Доля населения с высшим образованием, %',
    'degree': u'Доля населения, имеющего ученые степени, %',
    'ict': u'Доля специалистов в области ИКТ, %',
    'grp': u'ВРП на душу населения, рублей',
    'income': u'Среднедушевые доходы населения, рублей',
    'po': u'Объем использования программного обеспечения, %',
    'invest': u'Удельный вес инвестиций в основной капитал в ВВП, %',
    'venture': u'Доступность венчурного капитала, штук',
    'pc': u'Количество персональных компьютеров на 100 человек, штук',
    'nt': u'Затраты организаций на сетевые технологии, миллионов рублей',
    'ict_grp': u'Удельный вес затрат на ИКТ в ВРП, %',
}

Считываем данные из соответствующих файлов.

In [ ]:
institutional_factors = pd.read_csv('institutional_factors.csv', index_col='index')
institutional_factors.df_name = u'Институциональные факторы'

infrastructural_factors = pd.read_csv('infrastructural_factors.csv', index_col='index')
infrastructural_factors.df_name = u'Инфраструктурные факторы'

resource_factors = pd.read_csv('resource_factors.csv', index_col='index')
resource_factors.df_name = u'Ресурсные факторы'

factors = [institutional_factors, infrastructural_factors, resource_factors]
targets = pd.read_csv('target.csv', index_col='index')

Изучим характеристики таблиц факторов.

In [ ]:
for df in factors:
    print '\n' + df.df_name.upper()
    display(df.describe())

Некоторые индикаторы принимают лишь одно значение на изучаемой выборке и бесполезны для построения модели. Удалим их.

In [ ]:
cleared_factors = []
for df in factors:
    for column in df.columns:
        if df.min()[column] == df.max()[column]:
            name = df.df_name
            df = df.drop(column, axis=1)
            df.df_name = name
    cleared_factors.append(df)
for df in cleared_factors:
    print '\n' + df.df_name.upper()
    display(df[:10])

Теперь взглянем на корреляции факторов в каждой группе.

In [ ]:
for df in cleared_factors:
    f, ax = plt.subplots(figsize=(15, 15))
    sb.plt.title('\n' + df.df_name.upper())
    sb.corrplot(df)
#     corr = np.corrcoef(df)
#     mask = np.zeros_like(corr)
#     mask[np.triu_indices_from(mask)] = True
#     with sb.axes_style("white"):
#         ax = sb.heatmap(corr, mask=mask)


Некоторые факторы тесно коррелируют друг с другом (> CORRELLATION_LIMIT по модулю). Посмотрим на них поближе.

In [ ]:
correlated = {df.df_name: [] for df in cleared_factors}
for df in cleared_factors:
    correlation = df.corr()
    for c1 in df.columns:
        for c2 in df.columns:
            if abs(correlation[c1][c2]) > CORRELLATION_LIMIT and c1 != c2:
                if [c2, c1, correlation[c1][c2]] not in correlated[df.df_name]:
                    correlated[df.df_name].append([c1, c2, correlation[c1][c2]])
for df_name, items in correlated.iteritems():
    if correlated[df_name]: print '\n' + df_name
    for item in items:
        print item

Из каждой группы удалим минимально необходимое для устранения тесных корреляций число факторов.

In [ ]:
uncorrelated_factors = []
for df in cleared_factors:
    while correlated[df.df_name]:
        frequencies, removed = {}, []
        for item in correlated[df.df_name]:
            if item[0] in frequencies and item[1] in frequencies:
                frequencies[item[0]] += 1
                frequencies[item[1]] += 1
            elif item[0] in frequencies and item[1] not in frequencies:
                frequencies[item[0]] += 1
                frequencies[item[1]] = 1
            elif item[0] not in frequencies and item[1] in frequencies:
                frequencies[item[0]] = 1
                frequencies[item[1]] += 1
            else:
                frequencies[item[0]] = 1
                frequencies[item[1]] = 1
        most_frequent = max(frequencies, key=lambda i: frequencies[i])
        for item in list(correlated[df.df_name]):
            if most_frequent in item:
                correlated[df.df_name].remove(item)
        name = df.df_name
        df = df.drop(most_frequent, axis=1)
        df.df_name = name
    uncorrelated_factors.append(df)
for df in uncorrelated_factors:
    print '\n' + df.df_name.upper()
    display(df)

Для каждого набора по оставшимся факторам отберем наилучшие модели по качеству предсказания бинарных признаков.

In [ ]:
target_groups = targets.columns.to_series().groupby(targets.dtypes).groups
categorial_targets = [sublist for k, v in target_groups.iteritems() if k.name not in NUMERIC_TYPES for sublist in v]
print u'Бинарные целевые переменные: \n'
for ct in categorial_targets: print ct
print u'\n\nНепрерывные целевые переменные: \n'
numeric_targets = [nt for nt in set(targets.columns) - set(categorial_targets)]
for nt in numeric_targets: print nt

In [ ]:
kf = cross_validation.KFold(len(targets), n_folds=3, shuffle=True)  # используем это разбиение для всех методов
clfs = CLASSIFIERS
c_values, n_trees = np.logspace(-4, 1, 6), np.arange(5, 50, 6) 
best_rez_clf = {tname: {fs.df_name: {'score': -999999} for fs in uncorrelated_factors} for tname in categorial_targets}
for target_name in categorial_targets:
    for factors in uncorrelated_factors:
        target = targets.loc[:, target_name]
        for classifier in clfs:
            for c, n in zip(c_values, n_trees):
#                 start_time = datetime.datetime.now()
                try:
                    clf = classifier(C=c).fit(factors, target)
                except TypeError:
                    clf = classifier(n_estimators=n).fit(factors, target)
                score_list = cross_validation.cross_val_score(clf, factors, target, scoring=SCORING_CLF, cv=kf)
                average_val_score = sum(score_list) / float(len(score_list))
#                 end_time = datetime.datetime.now() - start_time
#                 hours, remainder = divmod(end_time.total_seconds() , 3600)
#                 minutes, seconds = divmod(remainder, 60)
#                 print '%s, %s, var = %.5f, av_score = %.5f, time = %s:%s' % \
#                         (factors.df_name, classifier.__name__, c, average_val_score, minutes, seconds)
                if best_rez_clf[target_name][factors.df_name]['score'] < average_val_score:
                    best_rez_clf[target_name][factors.df_name]['score'] = average_val_score
                    best_rez_clf[target_name][factors.df_name]['var'] = c
                    if classifier.__name__ == 'GradientBoostingClassifier': 
                        best_rez_clf[target_name][factors.df_name]['var'] = n
                    best_rez_clf[target_name][factors.df_name]['clf'] = clf
                    # best_rez_clf[target_name][factors.df_name]['coef'] = clf.coef_
for target in best_rez_clf:
    print target
    for fname in best_rez_clf[target]:
        print '\t' + fname
        for k, v in best_rez_clf[target][fname].iteritems():
            print '\t\t' + k + ': ' + str(v)[:60]
            
    print ''

In [ ]:
for target_name in categorial_targets:
    for factors in uncorrelated_factors:
        scfac = preprocessing.scale(factors)
        target = targets.loc[:, target_name]
        for classifier in [clfs[0]]:
                c = 0.01
#                 start_time = datetime.datetime.now()
                try:
                    clf = classifier(C=c).fit(scfac, target)
                except TypeError:
                    clf = classifier(n_estimators=n).fit(scfac, target)
                print factors.df_name
                print target_name    
                print '----------------------------------------'
                print 'Column names:'
                print list(factors.columns)
                print '----------------------------------------'
                print 'Coefficients:'
                print clf.coef_
                print '----------------------------------------'
#                 print 'Means:'
#                 print scfac.mean()
#                 print '----------------------------------------'
#                 print 'Coef * Mean:'
#                 print scfac.mean()*clf.coef_[0]
#                 print '----------------------------------------'
                print 'Full names:'
                for col_name in list(factors.columns):
                    print col_name, '-', FACTOR_NAMES[col_name]
                print ''
                print '========================================'
                

print '========================================'
print ''

for target_name in numeric_targets:
    for factors in uncorrelated_factors:
        target = targets.loc[:, target_name]
        for classifier in [linear_model.LinearRegression]:
                clf = classifier().fit(factors, target)
                print factors.df_name
                print target_name    
                print '----------------------------------------'
                print 'Column names:'
                print list(factors.columns)
                print '----------------------------------------'
                print 'Coefficients:'
                print clf.coef_
                print '----------------------------------------'
                print 'Means:'
                print factors.mean()
                print '----------------------------------------'
                print 'Coef * Mean:'
                print factors.mean()*clf.coef_
                print '----------------------------------------'
                print 'Full names:'
                for col_name in list(factors.columns):
                    print col_name, '-', FACTOR_NAMES[col_name]
                print ''
                print '========================================'
    print '========================================'
    print ''

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

for target_name in categorial_targets:
    for factors in uncorrelated_factors:
        target = targets.loc[:, target_name]
        for classifier in [clfs[0]]:
                c = 0.01
#                 start_time = datetime.datetime.now()
                try:
                    clf = classifier(C=c).fit(scfac, target)
                except TypeError:
                    clf = classifier(n_estimators=n).fit(scfac, target)
                print factors.df_name
                print target_name    
                print '----------------------------------------'
                print 'Column names:'
                print list(factors.columns)
                print '----------------------------------------'

                #rank all features, i.e continue the elimination until the last one
                rfe = RFE(clf, n_features_to_select=1)
                rfe.fit(factors,target)

                print "Features sorted by their rank:"  
                print sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), list(factors.columns)))
                print 'Selected features:'
                print rfe.n_features_
                print ''
                print '========================================'
                

print '========================================'
print ''

for target_name in numeric_targets:
    for factors in uncorrelated_factors:
        target = targets.loc[:, target_name]
        for classifier in [linear_model.LinearRegression]:
                clf = classifier().fit(factors, target)
                print factors.df_name
                print target_name    
                print '----------------------------------------'
                print 'Column names:'
                print list(factors.columns)
                print '----------------------------------------'

                #rank all features, i.e continue the elimination until the last one
                rfe = RFE(clf, n_features_to_select=1)
                rfe.fit(factors,target)

                print "Features sorted by their rank:"  
                print sorted(zip(map(lambda x: round(x, 4), rfe.ranking_), list(factors.columns)))
                print 'Selected features:'
                print rfe.n_features_
                print ''
                print '========================================'
    print '========================================'
    print ''

Теперь для каждого набора построим модели для предсказания непрерывных целевых переменных.

In [ ]:
clfs = REGRESSORS
alphas, n_trees = np.logspace(-5, -1, 5), np.arange(5, 50, 6) 
best_rez_rgr = {tname: {fs.df_name: {'score': -999999} for fs in uncorrelated_factors} for tname in numeric_targets}
for target_name in numeric_targets:
    for factors in uncorrelated_factors:
        target = targets.loc[:, target_name]
        for classifier in clfs:
            for a, n in zip(alphas, n_trees):
#                 start_time = datetime.datetime.now()
                try:
                    clf = classifier(alpha=a).fit(factors, target)
                except TypeError:
                    clf = classifier(n_estimators=n).fit(factors, target)
                # r2_score = 1 - residual sum of square / total sum of squares
                score_list = cross_validation.cross_val_score(clf, factors, target, scoring=SCORING_RGR, cv=kf)
                average_val_score = sum(score_list) / float(len(score_list))
#                 end_time = datetime.datetime.now() - start_time
#                 hours, remainder = divmod(end_time.total_seconds() , 3600)
#                 minutes, seconds = divmod(remainder, 60)
#                 print '%s, %s, var = %.5f, av_score = %.5f, time = %s:%s' % \
#                         (factors.df_name, classifier.__name__, c, average_val_score, minutes, seconds)
                if best_rez_rgr[target_name][factors.df_name]['score'] < average_val_score:
                    best_rez_rgr[target_name][factors.df_name]['score'] = average_val_score
                    best_rez_rgr[target_name][factors.df_name]['var'] = a
                    if classifier.__name__ == 'GradientBoostingClassifier': 
                        best_rez_rgr[target_name][factors.df_name]['var'] = n
                    best_rez_rgr[target_name][factors.df_name]['clf'] = clf
                    # best_rez_rgr[target_name][factors.df_name]['coef'] = clf.coef_
for target in best_rez_rgr:
    print target
    for fname in best_rez_rgr[target]:
        print '\t' + fname
        for k, v in best_rez_rgr[target][fname].iteritems():
            print '\t\t' + k + ': ' + str(v)[:60]
            
    print ''

Объединим факторы в один набор и посмотрим на корреляцию.

In [ ]:
all_factors = pd.concat(uncorrelated_factors, axis=1)
print all_factors
f, ax = plt.subplots(figsize=(20, 20))
sb.plt.title(u'Все факторы')
sb.corrplot(all_factors)

In [ ]:
all_correlated = []
correlation = all_factors.corr()
for c1 in all_factors.columns:
    for c2 in all_factors.columns:
        if abs(correlation[c1][c2]) > CORRELLATION_LIMIT and c1 != c2:
            if [c2, c1, correlation[c1][c2]] not in all_correlated:
                all_correlated.append([c1, c2, correlation[c1][c2]])
for item in all_correlated:
    print item

Удалим минимально необходимое для устранения тесных корреляций число факторов.

In [ ]:
all_uncorrelated, removed = [], []
while all_correlated:
    frequencies = {}
    for item in all_correlated:
        if item[0] in frequencies and item[1] in frequencies:
            frequencies[item[0]] += 1
            frequencies[item[1]] += 1
        elif item[0] in frequencies and item[1] not in frequencies:
            frequencies[item[0]] += 1
            frequencies[item[1]] = 1
        elif item[0] not in frequencies and item[1] in frequencies:
            frequencies[item[0]] = 1
            frequencies[item[1]] += 1
        else:
            frequencies[item[0]] = 1
            frequencies[item[1]] = 1
    most_frequent = max(frequencies, key=lambda i: frequencies[i] if i not in removed else 0)
    removed.append(most_frequent)
    for item in all_correlated:
        if most_frequent in item:
            all_correlated.remove(item)
    all_factors = all_factors.drop(most_frequent, axis=1)
display(all_factors)

Число факторов слишком велико для имеющегося количества наблюдений. Снова разобьем данные на три части, соответствующие первоначальным наборам, понизим размерность с помощью метода главных компонент для каждого из наборов до двух.

In [ ]:
resource_columns = [c for c in resource_factors.columns if c in all_factors.columns]
infrastractural_columns = [c for c in infrastructural_factors.columns if c in all_factors.columns]
institutional_columns = [c for c in institutional_factors.columns if c in all_factors.columns]

res = all_factors[resource_columns]
res_scaled = preprocessing.scale(res)
res_pca = decomposition.PCA(n_components=2)
resource_pca = pd.DataFrame(res_pca.fit_transform(res_scaled), 
                            index=resource_factors.index, 
                            columns=['res0', 'res1'])

inf = all_factors[infrastractural_columns]
inf_scaled = preprocessing.scale(inf)
inf_pca = decomposition.PCA(n_components=2)
infrastructural_pca = pd.DataFrame(inf_pca.fit_transform(inf_scaled), 
                                   index=resource_factors.index, 
                                   columns=['inf0', 'inf1'])

ins = all_factors[institutional_columns]
ins_scaled = preprocessing.scale(ins)
ins_pca = decomposition.PCA(n_components=2)
institutional_pca = pd.DataFrame(ins_pca.fit_transform(ins_scaled), 
                                 index=resource_factors.index,
                                 columns=['ins0', 'ins1'])

Вновь объединим наборы и приступим к построению моделей.

In [ ]:
all_factors_pca = pd.concat([resource_pca, infrastructural_pca, institutional_pca], axis=1)
all_factors_pca

In [ ]:
clfs = CLASSIFIERS
c_values, n_trees = np.logspace(-4, 1, 6), np.arange(5, 50, 6) 
pca_rez_clf = {tname: {'score': -999999} for tname in categorial_targets}
for tname in categorial_targets:
    target = targets.loc[:, tname]
    for classifier in clfs:
        for c, n in zip(c_values, n_trees):
#                 start_time = datetime.datetime.now()
            try:
                clf = classifier(C=c).fit(all_factors_pca, target)
            except TypeError:
                clf = classifier(n_estimators=n).fit(all_factors_pca, target)
            score_list = cross_validation.cross_val_score(clf, all_factors_pca, target, scoring=SCORING_CLF, cv=kf)
            average_val_score = sum(score_list) / float(len(score_list))
#                 end_time = datetime.datetime.now() - start_time
#                 hours, remainder = divmod(end_time.total_seconds() , 3600)
#                 minutes, seconds = divmod(remainder, 60)
#                 print '%s, %s, var = %.5f, av_score = %.5f, time = %s:%s' % \
#                         (factors.df_name, classifier.__name__, c, average_val_score, minutes, seconds)
            if pca_rez_clf[tname]['score'] < average_val_score:
                pca_rez_clf[tname]['score'] = average_val_score
                pca_rez_clf[tname]['var'] = c
                if classifier.__name__ == 'GradientBoostingClassifier': 
                    pca_rez_clf[tname]['var'] = n
                pca_rez_clf[tname]['clf'] = clf
                # best_rez_clf[target_name][factors.df_name]['coef'] = clf.coef_
for target in pca_rez_clf:
    print target
    for k, v in pca_rez_clf[target].iteritems():
        print '\t' + k + ': ' + str(v)[:60]
    print ''

In [ ]:
clfs = REGRESSORS
alphas, n_trees = np.logspace(-5, -1, 5), np.arange(5, 50, 6) 
pca_rez_rgr = {tname: {'score': -999999} for tname in numeric_targets}
for target_name in numeric_targets:
    target = targets.loc[:, target_name]
    for classifier in clfs:
        for a, n in zip(alphas, n_trees):
#                 start_time = datetime.datetime.now()
            try:
                clf = classifier(alpha=a).fit(all_factors_pca, target)
            except TypeError:
                clf = classifier(n_estimators=n).fit(all_factors_pca, target)
            # r2_score = 1 - residual sum of square / total sum of squares
            score_list = cross_validation.cross_val_score(clf, all_factors_pca, target, scoring=SCORING_RGR, cv=kf)
            average_val_score = sum(score_list) / float(len(score_list))
#                 end_time = datetime.datetime.now() - start_time
#                 hours, remainder = divmod(end_time.total_seconds() , 3600)
#                 minutes, seconds = divmod(remainder, 60)
#                 print '%s, %s, var = %.5f, av_score = %.5f, time = %s:%s' % \
#                         (factors.df_name, classifier.__name__, c, average_val_score, minutes, seconds)
            if pca_rez_rgr[target_name]['score'] < average_val_score:
                pca_rez_rgr[target_name]['score'] = average_val_score
                if classifier.__name__ == 'Ridge':
                    pca_rez_rgr[target_name]['var'] = a
                if classifier.__name__ == 'GradientBoostingRegressor': 
                    pca_rez_rgr[target_name]['var'] = n
                pca_rez_rgr[target_name]['clf'] = clf
                # best_rez_rgr[target_name][factors.df_name]['coef'] = clf.coef_
for target in pca_rez_rgr:
    print target
    for k, v in pca_rez_rgr[target].iteritems():
        print '\t' + k + ': ' + str(v)[:60]
            
    print ''

In [ ]:
from sklearn import tree
from IPython.display import Image
dot_data = tree.export_graphviz(pca_rez_clf['public_discussion']['clf'], out_file=None,
                                feature_names=all_factors_pca.columns,  
                                class_names=['0','1'], 
                                filled=True, rounded=True,  
                                special_characters=True) 